In [1]:
# This is a weird hack to avoid Intel MKL issues on the cluster when this is called as a subprocess of a process that has itself initialized PyTorch.
# Since numpy gets imported later anyway for dataset stuff, this shouldn't affect performance.
import numpy as np  # noqa: F401


# Create the TorchForce from the serialized compute graph
import sys
from openmm.app import PDBFile, ForceField, Simulation
from openmm.app import Modeller
from openmm.app import *
from openmm import *
from openmm.unit import kelvin, picosecond, femtosecond, nanometers

In [2]:
# PDB
# '/storage_common/angiod/CHIGNOLIN/chignolin.pdb' '/storage_common/angiod/chignolin_unfolded/chignolin.pdb'
input_file = '/storage_common/angiod/chignolin_unfolded/chignolin.pdb'

In [3]:
# load input molecule and initialize model
assert input_file.endswith('.pdb')
input = PDBFile(input_file)

# Create a system from the PDB file
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
# Add water molecules to the PDB structure

# Create a Modeller object
modeller = Modeller(input.topology, input.positions)
modeller.addExtraParticles(forcefield)

input.topology  = modeller.topology
input.positions = modeller.positions

In [4]:
for i in range(20, 21):
    system = forcefield.createSystem(input.topology)

    # while system.getNumForces() > 0:
    #     system.removeForce(0)

    # Construct using a serialized module:
    # torch_force.setOutputsForces(True)

    # # or using an instance of the module:
    # torch_force = TorchForce(module)

    # Add the TorchForce to your System
    # system.addForce(torch_force)

    # Create an integrator with a time step of 1 fs
    temperature = 310 * kelvin # 298.15 * kelvin
    frictionCoeff = 1.0 / picosecond # .1 / picosecond
    timeStep = 2 * femtosecond
    integrator = LangevinIntegrator(temperature, frictionCoeff, timeStep)
    # integrator = VerletIntegrator(timeStep)

    # Create a simulation and set the initial positions and velocities
    platform = Platform.getPlatformByName('CUDA') #'CUDA'
    simulation = Simulation(input.topology, system, integrator, platform=platform)
    simulation.context.setPositions(input.positions)

    simulation.reporters.append(PDBReporter(f'u2f.base.{i}.pdb', 100))
    simulation.reporters.append(StateDataReporter(sys.stdout, 100, step=True,potentialEnergy=True, temperature=True))

    simulation.context.setVelocitiesToTemperature(temperature)
    simulation.step(100000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,466.5496492385864,361.1690766223538
200,505.5409073829651,322.86931205401385
300,464.44296884536743,359.8496873337525
400,352.0958995819092,395.82708064926953
500,383.8564281463623,360.0779460852473
600,334.6566438674927,374.10935312470093
700,335.71637773513794,348.16842529448286
800,272.7245578765869,342.568817134296
900,252.71359872817993,365.2829251848706
1000,205.71990203857422,322.4529797764964
1100,143.27766799926758,356.93057610999483
1200,180.19193077087402,353.33484172672047
1300,177.81707000732422,337.48218625931025
1400,86.54650402069092,327.0368674191667
1500,109.79829597473145,321.32925765353934
1600,99.89366817474365,305.19786738343447
1700,62.269341468811035,334.56720343367067
1800,103.41432476043701,316.026494957571
1900,169.83911037445068,305.9746113914494
2000,63.78170967102051,309.3818843709454
2100,125.46615028381348,306.2477466804599
2200,79.2917308807373,289.2996496003757
2300,46.6591911315918,302.3769758

In [ ]:
# simulation.minimizeEnergy()

simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(100000)